In [ ]:
# default_exp dbt_cellmagic

# DBT Cell Magic 

> provides the `%%dbt` cell magic for creating models and analyses 

In [ ]:
# hide
# notest
%reload_ext autoreload
%reload_ext lab_black
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# exporti

import IPython
from IPython.core import magic_arguments
from IPython.core.magic import register_cell_magic, register_line_magic

In [ ]:
# export
from pathlib import Path


def write_sql(sql: str, file: str, project_dir: str):
    path = Path(project_dir) / file
    with open(path, "w") as f:
        f.write(sql)

In [ ]:
# export
from dbt.main import parse_args, adapter_management
from dbt.task.compile import CompileTask
import dbt.flags
import dbt.tracking

In [ ]:
# exporti
dbt.tracking.active_user = dbt.tracking.User(None)
dbt.flags.INDIRECT_SELECTION = "eager"
dbt.flags.WRITE_JSON = True

In [ ]:
# export
def compile_model(project, model):
    parsed = parse_args(["compile", "--select", model, "--project-dir", project])
    with adapter_management():
        task = CompileTask.from_args(args=parsed)
        # set_trace()
        op_result = task.run()
        return op_result

In [ ]:
# export
@magic_arguments.magic_arguments()
@magic_arguments.argument(
    "-a",
    "--assign",
    type=str,
    default=None,
    help=("If provided, save the output to this variable instead of displaying it."),
)
@magic_arguments.argument(
    "-p",
    "--project",
    type=str,
    help=("dbt project directory"),
)
@magic_arguments.argument(
    "-n",
    "--notebook",
    type=str,
    default=None,
    help=("notebook source file"),
)
@magic_arguments.argument(
    "file",
    type=str,
    help=("file path to write to"),
)
@register_cell_magic("dbt")
def write_dbt(line, cell):
    try:
        from dbt.main import parse_args
    except ImportError:
        return "'dbt-core' not installed. Did you run 'pip install dbt-core'?"
    line_args = magic_arguments.parse_argstring(write_dbt, line)
    print(f"assign={line_args.assign}")
    print(f"project={line_args.project}")
    print(f"notebook={line_args.notebook}")
    print(f"file={line_args.file}")

    if not line_args.notebook:
        contents = "-- AUTOGENERATED! DO NOT EDIT!\n" + cell
    else:
        contents = (
            f"-- AUTOGENERATED! DO NOT EDIT! File to edit: {line_args.notebook} (unless otherwise specified).\n"
            + cell
        )
    write_sql(contents, line_args.file, line_args.project)
    op_result = compile_model(line_args.project, line_args.file)
    results = op_result
    if line_args.assign:
        IPython.get_ipython().push({line_args.assign: results})
        return 0
    return results

In [ ]:
%%dbt -p ../my_dbt_project analyses/sample1.sql

select *
from {{ ref('my_first_dbt_model') }}
where id is not null

assign=None
project=../my_dbt_project
notebook=None
file=analyses/sample1.sql
04:45:35  Found 2 models, 4 tests, 0 snapshots, 3 analyses, 191 macros, 0 operations, 0 seed files, 0 sources, 0 exposures, 0 metrics
04:45:35  
04:45:38  Concurrency: 1 threads (target='dev')
04:45:38  
04:45:38  Done.


RunExecutionResult(results=[RunResult(status=<RunStatus.Success: 'success'>, timing=[TimingInfo(name='compile', started_at=datetime.datetime(2022, 6, 19, 4, 45, 38, 144157), completed_at=datetime.datetime(2022, 6, 19, 4, 45, 38, 149354)), TimingInfo(name='execute', started_at=datetime.datetime(2022, 6, 19, 4, 45, 38, 149762), completed_at=datetime.datetime(2022, 6, 19, 4, 45, 38, 149790))], thread_id='Thread-5', execution_time=0.007093191146850586, adapter_response={}, message=None, failures=None, node=CompiledAnalysisNode(raw_sql="-- AUTOGENERATED! DO NOT EDIT!\n\nselect *\nfrom {{ ref('my_first_dbt_model') }}\nwhere id is not null", compiled=True, database='sample-dbt-learn-project', schema='jaffle_shop', fqn=['my_dbt_project', 'analysis', 'sample1'], unique_id='analysis.my_dbt_project.sample1', package_name='my_dbt_project', root_path='/home/butch2/play/experiments/nbdbt/my_dbt_project', path='analysis/sample1.sql', original_file_path='analyses/sample1.sql', name='sample1', resource

In [ ]:
print(_)

RunExecutionResult(results=[RunResult(status=<RunStatus.Success: 'success'>, timing=[TimingInfo(name='compile', started_at=datetime.datetime(2022, 6, 19, 4, 45, 38, 144157), completed_at=datetime.datetime(2022, 6, 19, 4, 45, 38, 149354)), TimingInfo(name='execute', started_at=datetime.datetime(2022, 6, 19, 4, 45, 38, 149762), completed_at=datetime.datetime(2022, 6, 19, 4, 45, 38, 149790))], thread_id='Thread-5', execution_time=0.007093191146850586, adapter_response={}, message=None, failures=None, node=CompiledAnalysisNode(raw_sql="-- AUTOGENERATED! DO NOT EDIT!\n\nselect *\nfrom {{ ref('my_first_dbt_model') }}\nwhere id is not null", compiled=True, database='sample-dbt-learn-project', schema='jaffle_shop', fqn=['my_dbt_project', 'analysis', 'sample1'], unique_id='analysis.my_dbt_project.sample1', package_name='my_dbt_project', root_path='/home/butch2/play/experiments/nbdbt/my_dbt_project', path='analysis/sample1.sql', original_file_path='analyses/sample1.sql', name='sample1', resource

In [ ]:
%%dbt -a sample2 -p ../my_dbt_project -n notebooks/00_core.ipynb analyses/sample2.sql 

select *
from {{ ref('my_second_dbt_model') }}
where id is not null

assign=sample2
project=../my_dbt_project
notebook=notebooks/00_core.ipynb
file=analyses/sample2.sql
04:45:46  Found 2 models, 4 tests, 0 snapshots, 3 analyses, 191 macros, 0 operations, 0 seed files, 0 sources, 0 exposures, 0 metrics
04:45:46  
04:45:47  Concurrency: 1 threads (target='dev')
04:45:47  
04:45:47  Done.


0

In [ ]:
# notest
from nbdev.export import notebook2script

notebook2script("notebooks/00_core.ipynb")